In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/fetal-health-classification/fetal_health.csv')

In [ ]:
data.head()

In [ ]:
X = data.drop('fetal_health', axis=1)
y = pd.DataFrame(data['fetal_health'], columns=['fetal_health'])

In [ ]:
data.info()

In [ ]:
data.describe().T

In [ ]:
data.corr()['fetal_health'].drop('fetal_health').sort_values(ascending=False)

---

In [ ]:
hist = X.hist(figsize=(40, 30))

In [ ]:
# fetal_movement > .15,
# severe_decelerations > .0.0001
# mean_value_of_short_term_variabilty > 4.5
# percentage_of_time_with_abnormal_long_term_variablity > 70
# mean_value_of_long_term_variablity > 30
# histogram_max > 210
# histogram_number_of_zeroes > 4.2
# histogram_mode < 95
# histogram_variance > 160

In [ ]:
# len(list(X.loc[X['fetal_movement'] > .15, 'fetal_movement'].index))
X.loc[X['fetal_movement'] > .15, 'fetal_movement'] = X.loc[:, 'fetal_movement'].mean()

In [ ]:
# len(list(X.loc[X['mean_value_of_short_term_variability'] > 4.1, 'mean_value_of_short_term_variability'].index))
X.loc[X['mean_value_of_short_term_variability'] > 4.1, 'mean_value_of_short_term_variability'] = X.loc[:, 'mean_value_of_short_term_variability'].mean()

In [ ]:
# len(list(X.loc[X['percentage_of_time_with_abnormal_long_term_variability'] > 72, 'percentage_of_time_with_abnormal_long_term_variability'].index))
X.loc[X['percentage_of_time_with_abnormal_long_term_variability'] > 72, 'percentage_of_time_with_abnormal_long_term_variability'] = X.loc[:, 'percentage_of_time_with_abnormal_long_term_variability'].mean()

In [ ]:
# len(list(X.loc[X['mean_value_of_long_term_variability'] > 25.5, 'mean_value_of_long_term_variability'].index))
X.loc[X['mean_value_of_long_term_variability'] > 25.5, 'mean_value_of_long_term_variability'] = X.loc[:, 'mean_value_of_long_term_variability'].mean()

In [ ]:
# len(list(X.loc[X['histogram_max'] > 200, 'histogram_max'].index))
X.loc[X['histogram_max'] > 200, 'histogram_max'] = X.loc[:, 'histogram_max'].mean()

In [ ]:
# len(list(X.loc[X['histogram_number_of_zeroes'] > 2, 'histogram_number_of_zeroes'].index))
X.loc[X['histogram_number_of_zeroes'] > 2, 'histogram_number_of_zeroes'] = X.loc[:, 'histogram_number_of_zeroes'].mean()

In [ ]:
# len(list(X.loc[X['histogram_mode'] < 88, 'histogram_mode'].index))
X.loc[X['histogram_mode'] < 88, 'histogram_mode'] = X.loc[:, 'histogram_mode'].mean()

In [ ]:
# len(list(X.loc[X['histogram_variance'] > 118, 'histogram_variance'].index))
X.loc[X['histogram_variance'] > 118, 'histogram_variance'] = X.loc[:, 'histogram_variance'].mean()

---

In [ ]:
info = pd.concat([X, y], axis=1).corr()[y.columns[0]].drop(y.columns[0]).sort_values(ascending=False)
upper_columns = list(info[:9].index)
lower_columns = list(info[10:].index)

columns = list(X.columns)
from sklearn.preprocessing import MinMaxScaler
X = pd.DataFrame(MinMaxScaler(feature_range=(1, 3)).fit_transform(X), columns=columns) # to avoid x/0 or something nearly returns inf values

new_arrays = np.zeros([X.shape[0], 1])
for i in upper_columns:
    new_arrays = np.concatenate([new_arrays, (X[i] ** 2).values.reshape(-1, 1)], axis=1)
for i in upper_columns:
    new_arrays = np.concatenate([new_arrays, (X[i] ** 2).values.reshape(-1, 1)], axis=1)

for i in upper_columns:
    for j in lower_columns:
        new_arrays = np.concatenate([new_arrays, (X[i] / X[j]).values.reshape(-1, 1)], axis=1)
        new_arrays = np.concatenate([new_arrays, (X[j] / X[i]).values.reshape(-1, 1)], axis=1)
for i in upper_columns:
    for j in upper_columns:
        if i != j:
            new_arrays = np.concatenate([new_arrays, (X[i] * X[j]).values.reshape(-1, 1)], axis=1)
for i in lower_columns:
    for j in lower_columns:
        if i != j:
            new_arrays = np.concatenate([new_arrays, (X[i] * X[j]).values.reshape(-1, 1)], axis=1)
X = pd.concat([pd.DataFrame(X), pd.DataFrame(new_arrays[:, 1:])], axis=1)

In [ ]:
from sklearn.decomposition import KernelPCA
kpca = KernelPCA(kernel='rbf', n_components=round(X.shape[1]/6))
kpca.fit(X)
# X = kpca.transform(X) # if model overfits
X = np.concatenate([X, kpca.transform(X)], axis=1) # else

---

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=33)
X_smote, y_smote = smote.fit_resample(X, y)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_smote = scaler.fit_transform(X_smote)

In [ ]:
from sklearn.model_selection import train_test_split
Xsmtrain, Xsmvalid, ysmtrain, ysmvalid = train_test_split(X_smote, y_smote, test_size=.1, random_state=333)

---

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score

In [ ]:
from catboost import CatBoostClassifier as cbc
model = cbc(n_estimators=2500, verbose=0)
model.fit(Xsmtrain, ysmtrain.values.ravel())
ysmpred = model.predict(Xsmvalid)
accuracy_score(ysmvalid, ysmpred), confusion_matrix(ysmvalid, ysmpred)